In [ ]:
fig, ax = plt.subplots(len(data['type'].unique()), sharex=True, figsize=(15,10*len(data['type'].unique())))

k = 0
for i in data['type'].unique():
    data_4 = data[data['type'] == i]
    wordcloud = WordCloud().generate(data_4['posts'].to_string())
    ax[k].imshow(wordcloud)
    ax[k].set_title(i)
    ax[k].axis("off")
    k+=1

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from scipy.misc import imread
from wordcloud import WordCloud, STOPWORDS
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD

In [ ]:
data = pd.read_csv('../input/mbti_1.csv')
data.head()




In [ ]:
plt.figure(figsize=(40,20))
plt.xticks(fontsize=24, rotation=0)
plt.yticks(fontsize=24, rotation=0)
sns.countplot(data=data, x='type')

In [ ]:
data.groupby('type').agg({'type':'count'})

In [ ]:
data.iloc[1,1]

In [ ]:

data['postu_sk'] = data[['posts']].applymap(lambda x: str.count(x, '|||')+1)
data.head(20)

In [ ]:
data['post_http'] = data[['posts']].applymap(lambda x: str.count(x, 'http'))
data.head()

In [ ]:
data.groupby(['type']).agg({'post_http':[sum,np.mean]}).sort_values([('post_http', 'sum')])



In [ ]:
data['post_jpg'] = data[['posts']].applymap(lambda x: str.count(x, 'jpg'))
data.head()

In [ ]:
#A function is a block of code which only runs when it is called.

#You can pass data, known as parameters, into a function.

#A function can return data as a result

# string.split(s[, sep[, maxsplit]])
#Return a list of the words of the string s. If the optional second argument sep is absent or None, the words are separated 
#by arbitrary strings of whitespace characters (space, tab, newline, return, formfeed).

In [ ]:

def var_row(row):
    l = []
    for i in row.split('|||'):
        l.append(len(i.split()))
    return np.var(l)

data['zodziu_sk'] = data['posts'].apply(lambda x: len(x.split()))

data['zodziu_per_koment'] = data['zodziu_sk'] / data['postu_sk']

data.head()


In [ ]:

# Lemmatizer | Stemmatizer
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
unique_type_list = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
lab_encoder = LabelEncoder().fit(unique_type_list) #Encode labels with value between 0 and n_classes-1.
list_personality = []  

for i, row in data.iterrows():
    
    # One post
    OnePost = row.posts

    # Cache the stop words for speed 
    cachedStopWords = stopwords.words("english")
    
    #Return the string obtained by replacing the leftmost non-overlapping occurrences of pattern in string by the replacement repl. 
    #If the pattern isn’t found, string is returned unchanged.
    
    # List all urls
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', OnePost)

    # Remove urls
    temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'link', OnePost)

    # Keep only words
    temp = re.sub("[^a-zA-Z]", " ", temp)

    # Remove spaces > 1
    temp = re.sub(' +', ' ', temp).lower()
     #if remove_stop_words:
    temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in cachedStopWords])
       # else:
    #temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ')]) #Join all items in a tuple into a string,

    data.loc[i,'pak_tekstas'] = temp
    
    data.loc[i, 'pak_tipas'] = lab_encoder.transform([row.type])[0]
    type_labelized = lab_encoder.transform([row.type])[0]
    list_personality.append(type_labelized)
        


In [ ]:
data.head()

In [ ]:


data['pak_tekst_zodziu_sk'] = data['pak_tekstas'].apply(lambda x: len(x.split()))

data['pak_zodziu_per_koment'] = data['pak_tekst_zodziu_sk'] / data['postu_sk']

data.head(15)

In [ ]:
def get_types(row):
    t=row['type']

    I = 0; N = 0
    T = 0; J = 0
    
    if t[0] == 'I': I = 1
    elif t[0] == 'E': I = 0
    else: print('I-E incorrect')
        
    if t[1] == 'N': N = 1
    elif t[1] == 'S': N = 0
    else: print('N-S incorrect')
        
    if t[2] == 'T': T = 1
    elif t[2] == 'F': T = 0
    else: print('T-F incorrect')
        
    if t[3] == 'J': J = 1
    elif t[3] == 'P': J = 0
    else: print('J-P incorrect')
    return pd.Series( {'IE':I, 'NS':N , 'TF': T, 'JP': J }) 

data = data.join(data.apply (lambda row: get_types (row),axis=1))   #apply allow the users to pass a function and apply it on every single 
                                                                    #value of the Pandas series.
data.head(5)

In [ ]:
print ("Introversion (I) /  Extroversion (E):\t", data['IE'].value_counts()[0], " / ", data['IE'].value_counts()[1])
print ("Intuition (N) – Sensing (S):\t\t", data['NS'].value_counts()[0], " / ", data['NS'].value_counts()[1])
print ("Thinking (T) – Feeling (F):\t\t", data['TF'].value_counts()[0], " / ", data['TF'].value_counts()[1])
print ("Judging (J) – Perceiving (P):\t\t", data['JP'].value_counts()[0], " / ", data['JP'].value_counts()[1])
N = 4
bottom = (data['IE'].value_counts()[0], data['NS'].value_counts()[0], data['TF'].value_counts()[0], data['JP'].value_counts()[0])
top = (data['IE'].value_counts()[1], data['NS'].value_counts()[1], data['TF'].value_counts()[1], data['JP'].value_counts()[1])

ind = np.arange(N)    # grupiu vieta
width = 0.85      # platumas

p1 = plt.bar(ind, bottom, width)
p2 = plt.bar(ind, top, width, bottom)

plt.ylabel('Kiekis')
plt.xlabel('Tipai')
plt.title('Pasiskirstymas')

plt.xticks(ind, ('I/E',  'N/S', 'T/F', 'J/P',))

plt.show()

In [ ]:
data.groupby(['IE']).agg({'pak_tekst_zodziu_sk':[sum,np.mean]})

In [ ]:

cntizer = CountVectorizer(analyzer="word", 
                             max_features=1500, 
                             tokenizer=None,    
                             preprocessor=None, 
                             stop_words=None,  
#                             ngram_range=(1,1),
                             max_df=0.5,
                             min_df=0.1) 
                                 
tfizer = TfidfTransformer()

print("CountVectorizer")
X_cnt = cntizer.fit_transform(data['pak_tekstas'])
print("Tf-idf")
X_tfidf =  tfizer.fit_transform(X_cnt).toarray()


In [ ]:
X_tfidf


In [ ]:
X_tfidf.shape

In [ ]:
y = data['pak_tipas']
X = X_tfidf
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.

In [ ]:
clf = DecisionTreeClassifier(max_depth = 10)
clf.fit(X_train, y_train)
clf.predict(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset
and uses averaging to improve the predictive accuracy and control over-fitting. 

In [ ]:
for i in range(100,1100,100):
    for j in range(1,11,1):
        rfc = RandomForestClassifier(n_estimators=i, max_depth=j)

        rfc.fit(X_train,y_train)
#y_pred=rfc.predict(X_test)

        print(i,j, rfc.score(X_test, y_test))
    

In [ ]:

rfc = RandomForestClassifier(n_estimators=1000)

rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)

print(rfc.score(X_test, y_test))

In [ ]:

# Truncated SVD
svd = TruncatedSVD(n_components=12, n_iter=7, random_state=42)
svd_vec = svd.fit_transform(X_tfidf)

In [ ]:
y = data['IE']
X = svd_vec
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:

rfc = RandomForestClassifier(n_estimators=1000)

rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)

print(rfc.score(X_test, y_test))



In [ ]:
y = data['IE']
X = X_tfidf
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

rfc = RandomForestClassifier(n_estimators=1000)

rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)

print(rfc.score(X_test, y_test))

In [ ]:
y = data['NS']
X = svd_vec
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
rfc = RandomForestClassifier(n_estimators=1000)

rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)

print(rfc.score(X_test, y_test))

In [ ]:
y = data['TF']
X = svd_vec
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
rfc = RandomForestClassifier(n_estimators=1000)

rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)

print(rfc.score(X_test, y_test))

In [ ]:
y = data['JP']
X = svd_vec
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
rfc = RandomForestClassifier(n_estimators=1000)

rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)

print(rfc.score(X_test, y_test))

In [ ]:
y = data['JP']
X = X_tfidf
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
rfc = RandomForestClassifier(n_estimators=1000)

rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)

print(rfc.score(X_test, y_test))

In [ ]:
b_Pers = {'I':0, 'E':1, 'N':0, 'S':1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Pers_list = [{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]

def translate_personality(personality):
    # transform mbti to binary vector
    
    return [b_Pers[l] for l in personality]

def translate_back(personality):
    # transform binary vector to mbti personality
    
    s = ""
    for i, l in enumerate(personality):
        s += b_Pers_list[i][l]
    return s

# Check ...
d = data.head(4)
list_personality_bin = np.array([translate_personality(p) for p in d.type])
print("Binarize MBTI list: \n%s" % list_personality_bin)

In [ ]:
unique_type_list = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
  
unique_type_list = [x.lower() for x in unique_type_list]


# Lemmatize
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()

# Cache the stop words for speed 
cachedStopWords = stopwords.words("english")

def pre_process_data(data, remove_stop_words=True, remove_mbti_profiles=True):

    list_personality = []
    list_posts = []
    len_data = len(data)
    i=0
    
    for row in data.iterrows():
        i+=1
        if (i % 500 == 0 or i == 1 or i == len_data):
            print("%s of %s rows" % (i, len_data))
 ##### Remove and clean comments
        posts = row[1].posts
        temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', posts)
        temp = re.sub("[^a-zA-Z]", " ", temp)
        temp = re.sub(' +', ' ', temp).lower()
        if remove_stop_words:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in cachedStopWords])
        else:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ')])
            
        if remove_mbti_profiles:
            for t in unique_type_list:
                temp = temp.replace(t,"")

        type_labelized = translate_personality(row[1].type)
        list_personality.append(type_labelized)
        list_posts.append(temp)

    list_posts = np.array(list_posts)
    list_personality = np.array(list_personality)
    return list_posts, list_personality

In [ ]:
list_posts, list_personality  = pre_process_data(data, remove_stop_words=True)

In [ ]:
cntizer = CountVectorizer(analyzer="word", 
                             max_features=1500, 
                             tokenizer=None,    
                             preprocessor=None, 
                             stop_words=None,  
                             max_df=0.7,
                             min_df=0.1) 

# Learn the vocabulary dictionary and return term-document matrix
print("CountVectorizer...")
X_cnt = cntizer.fit_transform(list_posts)

# Transform the count matrix to a normalized tf or tf-idf representation
tfizer = TfidfTransformer()

print("Tf-idf...")
# Learn the idf vector (fit) and transform a count matrix to a tf-idf representation
X_tfidf =  tfizer.fit_transform(X_cnt).toarray()

In [ ]:
feature_names = list(enumerate(cntizer.get_feature_names()))
feature_names

In [ ]:
print("X: Posts in tf-idf representation \n* 1st row:\n%s" % X_tfidf[0])

In [ ]:
type_indicators = [ "IE: Introversion (I) / Extroversion (E)", "NS: Intuition (N) – Sensing (S)", 
                   "FT: Feeling (F) - Thinking (T)", "JP: Judging (J) – Perceiving (P)"  ]

for l in range(len(type_indicators)):
    print(type_indicators[l])

In [ ]:
# Posts in tf-idf representation
X = X_tfidf

# Let's train type indicator individually
for l in range(len(type_indicators)):
    print("%s ..." % (type_indicators[l]))
    
    # Let's train type indicator individually
    Y = list_personality[:,l]

    # split data into train and test sets
    seed = 7
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

    # fit model on training data
    model = XGBClassifier()
    model.fit(X_train, y_train)

    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print("* %s Accuracy: %.2f%%" % (type_indicators[l], accuracy * 100.0))